In [7]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from numpy.linalg import norm
# from numpy_ext import rolling_apply as rolling_apply_ext
tqdm.pandas()
import time

In [8]:
raw_df = pd.read_csv('/home/ec2-user/Capstone/1999-12-01-2023-12-31.csv')
# dates = pd.read_csv('sampled_sector_pairs_200.csv')['']

In [9]:
raw_df['Close'] = raw_df['Adj Close']

In [10]:
# print(raw_df.shape)
# raw_df = raw_df[raw_df.Date.isin(raw_df.Date.unique()[-3059:-3059+1259])]
# print(raw_df.shape)
# print(raw_df.Date.nunique())

In [11]:
raw_df

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,GICS Sector,GICS Sub-Industry
0,1999-12-01,47.343750,47.625000,46.781250,24.405205,24.405205,2062600.0,MMM,Industrials,Industrial Conglomerates
1,1999-12-02,47.500000,49.218750,47.468750,25.115868,25.115868,2617400.0,MMM,Industrials,Industrial Conglomerates
2,1999-12-03,48.750000,50.406250,48.750000,25.665030,25.665030,3060600.0,MMM,Industrials,Industrial Conglomerates
3,1999-12-06,49.250000,49.531250,48.687500,25.277390,25.277390,1282200.0,MMM,Industrials,Industrial Conglomerates
4,1999-12-07,48.750000,48.750000,46.500000,24.033714,24.033714,2190400.0,MMM,Industrials,Industrial Conglomerates
...,...,...,...,...,...,...,...,...,...,...
2701708,2023-12-22,195.320007,195.910004,192.740005,194.538773,194.538773,1548400.0,ZTS,Health Care,Pharmaceuticals
2701709,2023-12-26,194.880005,196.339996,194.089996,195.057587,195.057587,814600.0,ZTS,Health Care,Pharmaceuticals
2701710,2023-12-27,195.410004,197.009995,194.740005,196.454422,196.454422,766400.0,ZTS,Health Care,Pharmaceuticals
2701711,2023-12-28,197.619995,198.600006,196.529999,196.713837,196.713837,880100.0,ZTS,Health Care,Pharmaceuticals


In [12]:
p1 = raw_df[raw_df.Ticker=='META'][['Close','Ticker','Date']].reset_index(drop=True)
p2 = raw_df[raw_df.Ticker=='WAT'][['Close','Ticker','Date']].reset_index(drop=True)

In [13]:
df = pd.merge(p1,p2,how='left',on='Date',suffixes=['_P1','_P2'])
df.head()

,Close_P1,Ticker_P1,Date,Close_P2,Ticker_P2
0,38.230000,META,2012-05-18,80.809998,WAT
1,34.029999,META,2012-05-21,82.550003,WAT
2,31.000000,META,2012-05-22,82.769997,WAT
3,32.000000,META,2012-05-23,82.089996,WAT
4,33.029999,META,2012-05-24,82.250000,WAT


In [14]:
s = time.time()
df['abs_spread'] = (df['Close_P1'] - df['Close_P2']).abs()
df['abs_spread_mean'] = df.rolling(500).abs_spread.mean()
df['abs_spread_std'] = df.rolling(500).abs_spread.std()

df['abs_spread_mean_l28'] = df.rolling(28).abs_spread.mean()
df['abs_spread_std_l28'] = df.rolling(28).abs_spread.std()
df['spread_normed'] = (df['abs_spread']-df['abs_spread_mean'])/df['abs_spread_std']
df['abs_spread_normed_max'] = df.spread_normed.abs().rolling(500).max()
df['abs_spread_normed_90th'] = df.spread_normed.abs().rolling(500).quantile(0.9)
df['abs_spread_normed_75th'] = df.spread_normed.abs().rolling(500).quantile(0.75)
df['abs_spread_normed_median'] = df.spread_normed.abs().rolling(500).median()
df['abs_spread_normed_l7_avg'] = df.spread_normed.abs().rolling(7).mean()
df['abs_spread_normed_l14_avg'] = df.spread_normed.abs().rolling(14).mean()

e = time.time()
print(e-s)

df.head()

0.009425163269042969


,Close_P1,Ticker_P1,Date,Close_P2,Ticker_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_l28,abs_spread_std_l28,spread_normed,abs_spread_normed_max,abs_spread_normed_90th,abs_spread_normed_75th,abs_spread_normed_median,abs_spread_normed_l7_avg,abs_spread_normed_l14_avg
0,38.230000,META,2012-05-18,80.809998,WAT,42.579998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,34.029999,META,2012-05-21,82.550003,WAT,48.520004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31.000000,META,2012-05-22,82.769997,WAT,51.769997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,32.000000,META,2012-05-23,82.089996,WAT,50.089996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,33.029999,META,2012-05-24,82.250000,WAT,49.220001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
def cos_sim(rs,df):
    rows = df.loc[rs.index]
    vec1_sub1 = rows['Close_P1']
    vec2_sub1 = rows['Close_P2']
    return np.dot(vec1_sub1, vec2_sub1) / (norm(vec1_sub1) * norm(vec2_sub1))

In [16]:
df['cos_sim'] = df['Close_P1'].rolling(500).progress_apply(cos_sim, args=(df,))

0it [00:00, ?it/s]

In [17]:
df['forward_abs_spread'] = df['abs_spread'].shift(-1)

In [18]:
df['entry_thresh'] = df['abs_spread_mean'] + 1.5*df['abs_spread_std']
df['exit_thresh'] = df['abs_spread_mean'] + 0.1*df['abs_spread_std']

In [19]:
df['entry_signal'] = np.where(df['forward_abs_spread']>=df['entry_thresh'],1,0)
df['exit_signal'] = np.where(df['forward_abs_spread']<=df['exit_thresh'],1,0)

In [20]:
df['stock1_current_Close'] = df['Close_P1'].shift(-1)
df['stock2_current_Close'] = df['Close_P2'].shift(-1)
df['stock1_7_days_ago_Close'] = df['Close_P1'].shift(7)
df['stock2_7_days_ago_Close'] = df['Close_P2'].shift(7)
df['pct_delta_1'] = (df['stock1_current_Close']/df['stock1_7_days_ago_Close']) - 1
df['pct_delta_2'] = (df['stock2_current_Close']/df['stock2_7_days_ago_Close']) - 1
df['long_P1'] = np.where(df.pct_delta_1 >= df.pct_delta_2,0,1)

In [21]:
df.shape

(2923, 30)

In [22]:
def compute_pnl(rs,df):
    rows = df.loc[rs.index].iloc[::-1]
    
    long_P1 = rows.long_P1.iloc[0]

    rows['long_entry'] = np.where(rows.entry_signal==1, np.where(long_P1==1,rows.Close_P1,rows.Close_P2),np.nan)
    rows['long_exit'] = np.where(rows.exit_signal==1, np.where(long_P1==1,rows.Close_P1,rows.Close_P2),np.nan)
    rows['short_entry'] = np.where(rows.entry_signal==1, np.where(long_P1!=1,rows.Close_P1,rows.Close_P2),np.nan)
    rows['short_exit'] = np.where(rows.exit_signal==1, np.where(long_P1!=1,rows.Close_P1,rows.Close_P2),np.nan)

    rows['exit_signal_sum'] = rows.groupby('entry_signal').exit_signal.cumsum()

    rows['long_entry'] = rows['long_entry'].ffill()
    rows['short_entry'] = rows['short_entry'].ffill()

    rows['long_entry'] = np.where(rows['exit_signal_sum']==1,rows['long_entry'],np.nan)

    rows['long_pnl'] = (rows.long_exit - rows.long_entry)/rows.long_entry
    rows['short_pnl'] = (rows.short_entry - rows.short_exit)/rows.short_entry

    if rows[rows.exit_signal_sum==1].shape[0]>0:
        pnl = rows[rows.exit_signal_sum==1].long_pnl.cumsum().iloc[-1]+rows[rows.exit_signal_sum==1].short_pnl.cumsum().iloc[-1]
    else:
        pnl = np.nan
        
#     if rows.Date.iloc[-1]=='2014-10-06':
#         print(rows)
#         print(rows[rows.exit_signal_sum==1].long_pnl.cumsum().iloc[-1],rows[rows.exit_signal_sum==1].short_pnl.cumsum().iloc[-1])
#         print(pnl)
    return pnl/2.0

In [23]:
df['pnl'] = df['Close_P1'].iloc[::-1].rolling(120).progress_apply(compute_pnl, args=(df,))
# .shift(-119)
df['pnl'].dropna().describe(np.arange(0,1,0.1))

0it [00:00, ?it/s]

count    278.000000
mean       0.005434
std        0.145096
min       -0.270693
0%        -0.270693
10%       -0.270693
20%       -0.026144
30%       -0.026144
40%       -0.019928
50%        0.019928
60%        0.025194
70%        0.026144
80%        0.026144
90%        0.270693
max        0.270693
Name: pnl, dtype: float64

In [ ]:
Everything below is for debugging purpose only

In [24]:
df.dropna().sort_values('pnl',ascending=False)

,Close_P1,Ticker_P1,Date,Close_P2,Ticker_P2,abs_spread,abs_spread_mean,abs_spread_std,abs_spread_mean_l28,abs_spread_std_l28,...,entry_signal,exit_signal,stock1_current_Close,stock2_current_Close,stock1_7_days_ago_Close,stock2_7_days_ago_Close,pct_delta_1,pct_delta_2,long_P1,pnl
2646,111.440002,META,2022-11-22,336.970001,WAT,225.529999,68.565520,65.641371,192.657144,29.131237,...,1,0,112.239998,334.570007,113.019997,331.850006,-0.006901,0.008196,1,0.270693
2620,134.039993,META,2022-10-17,279.600006,WAT,145.560013,61.032260,58.191488,141.703572,5.505776,...,1,0,132.800003,283.940002,139.070007,287.549988,-0.045085,-0.012554,1,0.270693
2622,133.229996,META,2022-10-19,277.359985,WAT,144.129990,61.384839,58.448094,141.902857,5.730582,...,0,0,131.529999,275.010010,133.789993,272.260010,-0.016892,0.010101,1,0.270693
2623,131.529999,META,2022-10-20,275.010010,WAT,143.480011,61.558200,58.562903,141.775001,5.650638,...,1,0,130.009995,282.119995,128.539993,271.989990,0.011436,0.037244,1,0.270693
2624,130.009995,META,2022-10-21,282.119995,WAT,152.110001,61.743760,58.702652,141.964645,5.908741,...,1,0,129.720001,287.410004,127.500000,270.720001,0.017412,0.061650,1,0.270693
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2656,113.930000,META,2022-12-07,340.640015,WAT,226.710014,72.160740,69.043879,216.428930,7.468592,...,1,0,115.330002,343.040009,108.779999,331.600006,0.060213,0.034499,0,-0.270693
2657,115.330002,META,2022-12-08,343.040009,WAT,227.710007,72.549280,69.371388,217.294288,7.314817,...,1,0,115.900002,336.529999,109.459999,330.160004,0.058834,0.019294,0,-0.270693
2659,114.709999,META,2022-12-12,340.859985,WAT,226.149986,73.313840,69.975106,218.313215,7.077027,...,1,0,120.150002,344.619995,120.440002,346.929993,-0.002408,-0.006658,0,-0.270693
2662,116.150002,META,2022-12-15,342.429993,WAT,226.279991,74.509780,70.839846,219.871429,6.512679,...,1,0,119.430000,338.480011,114.120003,337.589996,0.046530,0.002636,0,-0.270693


In [26]:
200/5940

0.03367003367003367

In [25]:
n = 3686
3463,3429
df[['Date','stock1_current_Close','stock2_current_Close','Close_P1','Close_P2',\
    'pct_delta_1','pct_delta_2','entry_signal','exit_signal','long_P1']].loc[[2608]]
# .loc[[3686,3734]]
# .loc[n:n+50]
# .iloc[n:n+25]

,Date,stock1_current_Close,stock2_current_Close,Close_P1,Close_P2,pct_delta_1,pct_delta_2,entry_signal,exit_signal,long_P1
2608,2022-09-29,135.679993,269.529999,136.410004,274.559998,-0.071257,-0.044152,0,0,1


In [21]:
p1_entry = 105.178848
p1_exit = 103.102409

p2_entry = 65.048088
p2_exit = 75.290604

In [22]:
long_entry = p1_entry
long_exit = p1_exit
long = (long_exit - long_entry)/long_entry
long

-0.019741982722609852

In [23]:
short_entry = p2_entry
short_exit = p2_exit
short = (short_entry - short_exit)/short_entry
short

-0.15746067739915728

In [27]:
rows = df[['Date','stock1_current_Close','stock2_current_Close','Close_P1','Close_P2',\
    'pct_delta_1','pct_delta_2','entry_signal','exit_signal','long_P1']].loc[2608:2608+120]
# .loc[3785-120:3785]

long_P1 = rows.long_P1.iloc[0]

rows['long_entry'] = np.where(rows.entry_signal==1, np.where(long_P1==1,rows.Close_P1,rows.Close_P2),np.nan)
rows['long_exit'] = np.where(rows.exit_signal==1, np.where(long_P1==1,rows.Close_P1,rows.Close_P2),np.nan)
rows['short_entry'] = np.where(rows.entry_signal==1, np.where(long_P1!=1,rows.Close_P1,rows.Close_P2),np.nan)
rows['short_exit'] = np.where(rows.exit_signal==1, np.where(long_P1!=1,rows.Close_P1,rows.Close_P2),np.nan)

rows['exit_signal_sum'] = rows.groupby('entry_signal').exit_signal.cumsum()

rows['long_entry'] = rows['long_entry'].ffill()
rows['short_entry'] = rows['short_entry'].ffill()

rows['long_entry'] = np.where(rows['exit_signal_sum']==1,rows['long_entry'],np.nan)
rows['short_entry'] = np.where(rows['exit_signal_sum']==1,rows['short_entry'],np.nan)

rows['long_pnl'] = (rows.long_exit - rows.long_entry)/rows.long_entry
rows['short_pnl'] = (rows.short_entry - rows.short_exit)/rows.short_entry

pnl = rows[rows.exit_signal_sum==1].long_pnl.cumsum().iloc[-1]+rows[rows.exit_signal_sum==1].short_pnl.cumsum().iloc[-1]
print(pnl/2.0)
rows

0.2706926714050584


,Date,stock1_current_Close,stock2_current_Close,Close_P1,Close_P2,pct_delta_1,pct_delta_2,entry_signal,exit_signal,long_P1,long_entry,long_exit,short_entry,short_exit,exit_signal_sum,long_pnl,short_pnl
2608,2022-09-29,135.679993,269.529999,136.410004,274.559998,-0.071257,-0.044152,0,0,1,NaN,NaN,NaN,NaN,0,NaN,NaN
2609,2022-09-30,138.610001,277.350006,135.679993,269.529999,-0.024697,-0.012779,0,0,1,NaN,NaN,NaN,NaN,0,NaN,NaN
2610,2022-10-03,140.279999,288.200012,138.610001,277.350006,-0.017785,0.047543,1,0,1,NaN,NaN,NaN,NaN,0,NaN,NaN
2611,2022-10-04,138.979996,290.839996,140.279999,288.200012,-0.010185,0.055681,1,0,1,NaN,NaN,NaN,NaN,0,NaN,NaN
2612,2022-10-05,139.070007,287.549988,138.979996,290.839996,0.019799,0.050795,1,0,1,NaN,NaN,NaN,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2724,2023-03-17,197.809998,300.510010,195.610001,298.970001,0.069417,-0.030050,0,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN
2725,2023-03-20,202.160004,308.630005,197.809998,300.510010,0.112664,-0.002327,0,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN
2726,2023-03-21,199.809998,302.260010,202.160004,308.630005,0.113086,-0.008854,0,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN
2727,2023-03-22,204.279999,298.170013,199.809998,302.260010,0.129243,-0.026670,0,1,0,139.369995,199.809998,338.75,302.260010,1,0.433666,0.10772


In [25]:
rows.entry_signal.value_counts()
rows.exit_signal.value_counts()

exit_signal
0    117
1      4
Name: count, dtype: int64